In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
train_file = "Adult Census Income Binary Classification dataset.csv"
train = pd.read_csv(train_file,sep=', ', engine='python')
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
train.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

In [4]:
train_xcat = train.copy()
train_xcat['education'] = train['education'].astype('category').cat.codes.astype(int)
train_xcat['workclass'] = train['workclass'].astype('category').cat.codes.astype(int)
train_xcat['marital-status'] = train['marital-status'].astype('category').cat.codes.astype(int)
train_xcat['occupation'] = train['occupation'].astype('category').cat.codes.astype(int)
train_xcat['relationship'] = train['relationship'].astype('category').cat.codes.astype(int)
train_xcat['race'] = train['race'].astype('category').cat.codes.astype(int)
train_xcat['sex'] = train['sex'].astype('category').cat.codes.astype(int)
train_xcat['native-country'] = train['native-country'].astype('category').cat.codes.astype(int)
train_xcat['income'] = train['income'].astype('category').cat.codes.astype(int)

In [5]:
del train_xcat["fnlwgt"]
#del train_xcat["capital-gain"]
#del train_xcat["capital-loss"]

In [6]:
train_xcat.dtypes

age               int64
workclass         int64
education         int64
education-num     int64
marital-status    int64
occupation        int64
relationship      int64
race              int64
sex               int64
capital-gain      int64
capital-loss      int64
hours-per-week    int64
native-country    int64
income            int64
dtype: object

In [9]:
import keras.utils
train_x = train_xcat.loc[:, train_xcat.columns != 'income']
train_y = keras.utils.to_categorical(train_xcat["income"])

In [10]:
#Create a model see how good it is
model = Sequential() #Left to right model
model.add(Dense(input_dim=train_x.shape[1], #Dense is a fully interconnected layer
                units=64,
                kernel_initializer='normal',
                bias_initializer='zeros',
                activation='relu'))

for i in range(0, 2):
    model.add(Dense(units=12,
                    kernel_initializer='normal',
                    bias_initializer='zeros',
                    activation='tanh'))
    
#for i in range(0, 2):
#    model.add(Dense(units=125,
#                    kernel_initializer='normal',
#                    bias_initializer='zeros',
#                    activation='relu'))
    
#for i in range(0, 2):
#    model.add(Dense(units=125,
#                    kernel_initializer='normal',
#                    bias_initializer='zeros',
#                    activation='tanh'))
   # model.add(Dropout(0.2)) #randomly selected neurons are ignored during training
    
model.add(Dense(2, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(train_x, train_y, epochs=10, validation_split=0.01, verbose=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                896       
_________________________________________________________________
dense_2 (Dense)              (None, 12)                780       
_________________________________________________________________
dense_3 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 26        
Total params: 1,858
Trainable params: 1,858
Non-trainable params: 0
_________________________________________________________________
Train on 32235 samples, validate on 326 samples
Epoch 1/10
32235/32235 [==============================] - 2s 69us/step - loss: 0.4506 - accuracy: 0.7845 - val_loss: 0.3985 - val_accuracy: 0.8037
Epoch 2/10
32235/32235 [==============================]

In [11]:
from interpret.ext.blackbox import TabularExplainer

# "features" and "classes" fields are optional
explainer = TabularExplainer(model, 
                             train_x)

In [ ]:
#Run the explainer
#import warnings
#We get a lot of warning so filter them out for this
#warnings.filterwarnings('ignore')

global_explanation = explainer.explain_global(train_x)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \


In [15]:
#Get the global feature importance
global_explanation.get_feature_importance_dict()

{'relationship': 0.12176038607696114,
 'education-num': 0.0542258842685571,
 'capital-gain': 0.045273930411500435,
 'marital-status': 0.03493366680528666,
 'age': 0.0336454752898766,
 'native-country': 0.023524806706434517,
 'capital-loss': 0.017362224850541112,
 'hours-per-week': 0.01720776970437656,
 'education': 0.005569411200663546,
 'occupation': 0.005511727721455784,
 'sex': 0.005002393917800827,
 'workclass': 0.0044256180962575055,
 'race': 0.0021550064629539166}

In [16]:
##Warning - Additional setup needed to run in Jupyter labs
from interpret_community.widget import ExplanationDashboard

ExplanationDashboard(global_explanation, model, datasetX=train_x)

ValueError: Can't clean for JSON: array([0.44038454, 0.5596155 ], dtype=float32)